In [ ]:
import hmmlearn
from hmmlearn import base
from HMM import MultiThreadFit
import pickle
import numpy as np
from tqdm import tqdm
import europarl_dataloader as e_dl
from hmm_training import TbXMonitor
import sys

In [ ]:
DATASET_PATH = "output/europarl-v7.de-en.de.clean"
TRAIN_STEP_SIZE = 20
THRESHOLD = 4
MODEL_PATH = "output/tss20_th4_nSt200_nIt101.pkl"

In [ ]:
print(MODEL_PATH)
lines = e_dl.load_clean_dataset(DATASET_PATH)
testLines = lines[:4096]
trainLines = lines[4096::TRAIN_STEP_SIZE]
del lines

trainWordcount = e_dl.count_words(trainLines)
testWordcount = e_dl.count_words(testLines)
trainAlphabet = e_dl.create_alphabet(trainWordcount, threshold=THRESHOLD)
testAlphabet = e_dl.create_test_alphabet(trainAlphabet, testWordcount)

In [ ]:
# prepare tokens
# lines_X = [[[trainAlphabet[word]] for word in line if len(word)]
#            for line in tqdm(trainLines, desc="trainTokenSet") if
#            len(line) > 1 or (len(line) == 1 and len(line[0]))]
# len_X = [len(line) for line in lines_X]
# X = np.concatenate(lines_X)
# del trainLines  # free space

lines_Y = [[[testAlphabet[word]] for word in line if len(word)]
           for line in tqdm(testLines, desc="testTokenSet") if
           len(line) > 1 or (len(line) == 1 and len(line[0]))]
len_Y = [len(line) for line in lines_Y]
Y = np.concatenate(lines_Y)
del testLines  # free space

In [ ]:
with open(MODEL_PATH, 'rb') as file:
    model = pickle.load(file)
""" :type model: MultiThreadFit"""

model.transmat_orig = model.transmat_.copy()
model.startprob_orig = model.startprob_.copy()
model.emissionprob_orig = model.emissionprob_.copy()

In [ ]:
print(model.score(Y, len_Y))

In [ ]:
print(f"Max Thresholds:\ntransmat_ {model.transmat_orig.max(axis=0).min()}\nemissionprob_ {model.emissionprob_orig.max(axis=0).min()}")

In [ ]:
transmat_th = model.transmat_orig.max(axis=0).min() * 0.1
model.transmat_ = model.transmat_orig.copy()
model.transmat_[model.transmat_orig < transmat_th] = 0
# model.transmat_ = model.transmat_ ** 0.5
for i in range(model.transmat_.shape[0]):
    model.transmat_[i,:] /= model.transmat_[i,:].sum()

import matplotlib.pyplot as plt
plt.title("orig")
plt.imshow(model.transmat_orig)
plt.colorbar()
plt.show()

plt.title("red")
plt.imshow(model.transmat_)
plt.colorbar()
plt.show()

plt.title("diff")
plt.imshow(model.transmat_- model.transmat_orig)
plt.colorbar()
plt.show()
        

In [ ]:
startprob_th = model.startprob_orig.max() * 0.1
model.startprob_ = model.startprob_orig.copy()
model.startprob_[model.startprob_orig < startprob_th] = 0
# model.startprob_ = model.startprob_** 0.5
model.startprob_ /= model.startprob_.sum()

In [ ]:
emissionprob_th = model.emissionprob_orig.max(axis=0).min() * 0.1
model.emissionprob_ = model.emissionprob_orig.copy()
model.emissionprob_[model.emissionprob_orig < emissionprob_th] = 0
# model.emissionprob_ = model.emissionprob_ ** 0.5
for i in range(model.emissionprob_.shape[0]):
    model.emissionprob_[i,:] /= model.emissionprob_[i,:].sum()

In [ ]:
print(model.score(Y, len_Y))